In [1]:
import numpy as np
import random as rnd

In [2]:
# Simplifying Assumptions
# Clock Phase:
# 	Simple Bids Only - No All or Nothing, AON+ with backstop price, or Switch Bids
# 	Only one category of license - encumbered licenses will be handled by simple POP % discounting
# 	All Bidders can bid on All Licenses, and they all want to cover all territories
# 	No Spectrum Reserve
# 	No Credits
# 	Single Clock Stage with single Final Stage Rule
# 	No Missing Bids
# 	Simplified Eligibility: 1BU=1License

# Assignment Phase:
# 	No parcel grouping
# 	No concern for contiguity of assigned blocks (but assume that there would be contiguity)

In [73]:
# In a real round, all of the input in this cell would be read in from the blockchain
# This would only be done after a check to make sure that the round had been closed

# Processed bids from the previous round
processed=[[[6,10000],[6,6000],[5,6000],[3,4000]],\
          [[5,10000],[5,6000],[4,6000],[4,4000]],\
          [[3,10000],[3,6000],[7,6000],[6,4000]]]

# New bids this round
bidinput=[[[5,12000],[3,7200],[5,7200],[3,4800]],\
          [[5,11000],[3,7200],[4,7200],[3,4800]],\
          [[3,11000],[3,6500],[5,6500],[5,4500]]]

round=2
numbidders=3
numparcels=4

elig_percent=.9
clockincrease=.2

eligibility=[20,20,20]
bidders=['A-Telecom','B-Net','C-Mobile']
parcels=['Elbograd','Mudberg','Deserton','Phlimsk']
minimums=[10000,6000,6000,4000]
supply=[10,10,9,6]

In [74]:
#calculate demand per parcel from last round, and excess demand

demand=[0,0,0,0]
for n in range(numparcels):
    for m in range(numbidders):
        print(n, m, processed[m][n][0])
        demand[n] += processed[m][n][0]
        
print(demand)

excess = [d-supply[idx] for idx,d in enumerate(demand)]
print(excess)

0 0 6
0 1 5
0 2 3
1 0 6
1 1 5
1 2 3
2 0 5
2 1 4
2 2 7
3 0 3
3 1 4
3 2 6
[14, 14, 16, 13]
[4, 4, 7, 7]


In [75]:
bidinput[1]  # Bid input from bidder 1 (B-net) for this round

[[5, 11000], [3, 7200], [4, 7200], [3, 4800]]

In [76]:
bidinput[1][2]  # Bid input from bidder 1 (B-Net) on Parcel 2 (Deserton) for this round

[4, 7200]

In [77]:
print(bidinput[1][2][0])  # Number of Deserton licenses B-Net is bidding for in this round
print(bidinput[1][2][1])  # Bid price B-Net is offering for these licenses in this round

4
7200


In [78]:
if round==1:
    clock=minimums
else:
    clock = [int(x * (1+clockincrease)) for x in minimums]

In [79]:
clock

[12000, 7200, 7200, 4800]

In [80]:
# Insert some code to make sure none of the bids are outside the range from minimums to clock

In [81]:
bidinput[0]

[[5, 12000], [3, 7200], [5, 7200], [3, 4800]]

In [82]:
#Calculate pricepoints
ppoints = [[0,0,0,0],[0,0,0,0],[0,0,0,0]]
for b in range(numbidders):
    for p in range(numparcels):
        print(processed[b][p][1], bidinput[b][p][1], clock[p], \
              bidinput[b][p][1]-processed[b][p][1] , clock[p]-processed[b][p][1], \
              (bidinput[b][p][1]-processed[b][p][1])/(clock[p]-processed[b][p][1]))
        ppoints[b][p]=(bidinput[b][p][1]-processed[b][p][1])/(clock[p]-processed[b][p][1])

10000 12000 12000 2000 2000 1.0
6000 7200 7200 1200 1200 1.0
6000 7200 7200 1200 1200 1.0
4000 4800 4800 800 800 1.0
10000 11000 12000 1000 2000 0.5
6000 7200 7200 1200 1200 1.0
6000 7200 7200 1200 1200 1.0
4000 4800 4800 800 800 1.0
10000 11000 12000 1000 2000 0.5
6000 6500 7200 500 1200 0.4166666666666667
6000 6500 7200 500 1200 0.4166666666666667
4000 4500 4800 500 800 0.625


In [83]:
ppoints

[[1.0, 1.0, 1.0, 1.0],
 [0.5, 1.0, 1.0, 1.0],
 [0.5, 0.4166666666666667, 0.4166666666666667, 0.625]]

In [84]:
queue=[]  # create a queue of bids to be processed 

In [85]:
for b in range(numbidders):
    for p in range(numparcels):
        queue.append([ppoints[b][p], rnd.random(), b, eligibility[b], p, bidinput[b][p]])

In [86]:
queue.sort()     # sort the bid queue by pricepoint

#in a fully realized auction there would likely be a separate queue per parcel, but this way gives the same results

In [87]:
queue

[[0.4166666666666667, 0.7219035916536067, 2, 20, 2, [5, 6500]],
 [0.4166666666666667, 0.9291134574363628, 2, 20, 1, [3, 6500]],
 [0.5, 0.5533598603837604, 2, 20, 0, [3, 11000]],
 [0.5, 0.7237942850622697, 1, 20, 0, [5, 11000]],
 [0.625, 0.646930178665225, 2, 20, 3, [5, 4500]],
 [1.0, 0.08224692728308902, 1, 20, 2, [4, 7200]],
 [1.0, 0.0999077225605085, 1, 20, 3, [3, 4800]],
 [1.0, 0.1141673223077545, 0, 20, 1, [3, 7200]],
 [1.0, 0.4813049234428398, 0, 20, 2, [5, 7200]],
 [1.0, 0.6865190234910022, 1, 20, 1, [3, 7200]],
 [1.0, 0.9636380478578468, 0, 20, 0, [5, 12000]],
 [1.0, 0.9878193907658507, 0, 20, 3, [3, 4800]]]

In [88]:
##queue format:  0=pricepoint, 1=bidder, 2=eligibility, 3=parcel, 4[0]=bid qty, 4[1]=bid price

closing=minimums
new_elig=[e for e in eligibility]    
new_demand=[0,0,0,0]
newq=[]

# Add logic to run this queue loop until the newq is empty or a full pass runs with nothing processed

numprocessed = 1

while (len(queue) > 0 and numprocessed > 0):
    
    print('Starting queue pass. Eligibility', eligibility, new_elig)
    numprocessed=0
    print('Queue length', len(queue))    
    
    for q in queue:
        
        ppt=q[0]
        bdr=q[2]
        elg=q[3]
        pcl=q[4]
        qty=q[5][0]
        prc=q[5][1]
    
        
        print()
        
        print('bdr', bdr, 'pcl', pcl, 'procqty', processed[bdr][pcl][0],'newqty', qty, 'price',prc )
    
        # First check that the bid quantity is not greater than the bidder's current eligibility
        if qty <= new_elig[bdr]:
            
            
            #process bid if...
            # no change in demand
            if processed[bdr][pcl][0] == qty:
                print('no change in demand for this parcel by this bidder: will be fully processed')
            
                new_elig[bdr] -= qty  
                numprocessed +=1
                if prc > closing[pcl]: 
                    closing[pcl] = prc   
                
            # 
            
            elif processed[bdr][pcl][0] - qty <= excess[pcl]:
                print('reduction in demand is covered by excess demand: bid will be fully processed')
            
                excess[pcl] -= (processed[bdr][pcl][0] - qty)   # reduce excess demand for this parcel
                new_elig[bdr] -= qty   # reduce bidder's eligibility
                numprocessed +=1
                if prc > closing[pcl]:
                    closing[pcl] = prc  # raise the new closing price for the parcel if this bid is higher    
                #recalculated excess demand *and* decrement eligibility, updated posted price if needed
            
            elif processed[bdr][pcl][0] - qty > excess[pcl] and excess[pcl]>0: 
                print('reduction in demand is not fully covered by excess demand: bid will be partially processed')
            
                new_elig[bdr] -= excess[pcl]   # reduce bidder's eligibility
                
                excess[pcl] = 0   # reduce excess demand for this parcel
                numprocessed +=1
                if prc > closing[pcl]:
                    closing[pcl] = prc  # raise the new closing price for the parcel if this bid is higher 
                    
                newq.append(q)    
                    
                #recalculated excess demand, decrement eligibility, updated posted price if needed
        
            elif processed[bdr][pcl][0] - qty > excess[pcl] and excess[pcl]==0:
                print('reduction in demand with no excess demand: bid will not be processed')
            
                new_elig[bdr] -= excess[pcl]   # reduce bidder's eligibility
                
                excess[pcl] = 0   # reduce excess demand for this parcel
            
                newq.append(q)
                
                if prc > closing[pcl]:
                    closing[pcl] = prc  # raise the new closing price for the parcel if this bid is higher  
                
            # increase in demand is covered by bidder eligibility
        
            elif qty - processed[bdr][pcl][0]  < elg:
                print('increase in demand is covered by bidder eligibility')
        
                excess[pcl] += (qty - processed[bdr][pcl][0])   # increase excess demand for this parcel
                new_elig[bdr] -= qty   # reduce bidder's eligibility
                numprocessed +=1
                if prc > closing[pcl]:
                    closing[pcl] = prc  # raise the new closing price for the parcel if this bid is higher 
        
                #increased excess demand, decremented eligibility, updated posted price if needed
    
        else:
            print("Bidder's eligibility is less than bid quantity - bid not processed")
        
    
        #at end of loop
        #calculate next-round eligibility
       
        print()

    
    print('End of Queue Pass. Closing prices', closing, 'Demand', demand, 'Bids processed', numprocessed)
    print()
    
    queue=[r for r in newq]
    



Starting queue pass. Eligibility [20, 20, 20] [20, 20, 20]
Queue length 12

bdr 2 pcl 2 procqty 7 newqty 5 price 6500
reduction in demand is covered by excess demand: bid will be fully processed


bdr 2 pcl 1 procqty 3 newqty 3 price 6500
no change in demand for this parcel by this bidder: will be fully processed


bdr 2 pcl 0 procqty 3 newqty 3 price 11000
no change in demand for this parcel by this bidder: will be fully processed


bdr 1 pcl 0 procqty 5 newqty 5 price 11000
no change in demand for this parcel by this bidder: will be fully processed


bdr 2 pcl 3 procqty 6 newqty 5 price 4500
reduction in demand is covered by excess demand: bid will be fully processed


bdr 1 pcl 2 procqty 4 newqty 4 price 7200
no change in demand for this parcel by this bidder: will be fully processed


bdr 1 pcl 3 procqty 4 newqty 3 price 4800
reduction in demand is covered by excess demand: bid will be fully processed


bdr 0 pcl 1 procqty 6 newqty 3 price 7200
reduction in demand is covered by exc

In [89]:
#calculate new posted prices
#For each parcel:
#   If excess supply, then posted price = prev round clock price
#   If no excess supply, posted price is highest partially or fully processed price of a bid to reduce demand
#   i.e. the price at which a bid caused demand to equal supply
#   If no reduction in supply, same posted price as prev round

In [90]:
#output from q loop should inlude:
#   new eligibility for each bidder
#   posted price for each parcel
#   demand for each parcel

In [95]:

print('Output:\n')
print('Eligibility left over:', new_elig)
elig=[int(np.ceil((eligibility[idx]-new_elig[idx])/elig_percent)) for idx,e in enumerate(new_elig)]
print('Next round Eligibility:', elig)
print('Posted closing prices:', closing)
print('Demand:', demand)
if demand==supply: print('Demand equals Supply: Clock Phase Ends')

Output:

Eligibility left over: [4, 7, 4]
Next round Eligibility: [18, 15, 18]
Posted closing prices: [12000, 7200, 7200, 4800]
Demand: [14, 14, 16, 13]
